In [22]:
from jw_package import *
from functools import reduce

# 1. weather_v4
# 2. 전체 item, 한번이라도 팔린 item
# 3. 전체 store vs 각 store별

# 전체 item + 전체 store = 0.052
# 한번이라도 팔린 item + 전체 store = 0.075
# 전체 item + 각 store별 = 0.071
# 한번이라도 팔린 item + 각 store별 = 0.071

In [6]:
sales = pd.read_csv('../data/basic/train.csv')
keys = pd.read_csv('../data/basic/key.csv')
raw = pd.read_csv('../data/weather_v4_delete_nan.csv',index_col=0)

In [7]:
data = sales.merge(keys).merge(raw)

In [14]:
cols = list(data.columns[3:])
cols.remove('station_nbr')
cols.remove('sunset')
cols.remove('sunrise')
cols.remove('codesum')

In [15]:
cols

['units',
 'tmax',
 'tmin',
 'tavg',
 'depart',
 'dewpoint',
 'wetbulb',
 'heat',
 'cool',
 'snowfall',
 'preciptotal',
 'stnpressure',
 'sealevel',
 'resultspeed',
 'resultdir',
 'avgspeed']

In [16]:
formula = columns_to_relation(cols,None,'units',False)

In [17]:
formula

'units ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed'

In [18]:
results=[0,]
mean = 0
n = 0
for item_nbr in range(1,112):
    frame = data[data['item_nbr']==item_nbr]
    
    model = sm.OLS.from_formula(formula,data=frame)
    
    r2 = model.fit().rsquared
    
    if not np.isnan(r2):
#         print(item_nbr, r2)
        mean+=r2
        n+=1
    
    if r2>=0.3:
        print(item_nbr,r2)
    
    results.append(model)

d:\anaconda\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


37 0.343402842785
45 0.343709594869


In [19]:
mean/n

0.052193409068744728

In [23]:
no_zero = data.copy()
result=[]
stores = data['store_nbr'].unique()
print(stores)
for store in list(stores):
    item_nbr = data[(data['store_nbr']==store) & (data['units']!=0)]
    this_stores_items = item_nbr['item_nbr'].unique()
    print(store, this_stores_items)
    for each in list(this_stores_items):
        tok = no_zero[(no_zero['store_nbr']==store) & (no_zero['item_nbr']==each)]
        result.append(tok)
      
no_zero = reduce((lambda x,y : x.append(y)), result)

[ 2  6 38 42  7 13  8 12 43 16 19 21 29 33 30 36]
2 [ 5 44 11 85 93]
6 [  9  45  14 107]
38 [ 9 45 52 93 86 98]
42 [ 5 45 82]
7 [ 5  7 23 93 95]
13 [  5  36  46  93 104]
8 [  5  23  59  93  84 100]
12 [16 37 79 93 86]
43 [ 16  37  73  93   3 105]
16 [16 25 39 50 85 93 77 64]
19 [16 83 39 69 93 88 75 15]
21 [  5  44  91 109]
29 [ 8 41 45 56]
33 [ 9 44 85 92 93]
30 [16 44 49 51 65 13]
36 [ 9 44 30  1]


In [24]:
results=[0,]
mean = 0
n = 0
print(no_zero['item_nbr'].unique())
for item_nbr in no_zero['item_nbr'].unique():
    frame = no_zero[no_zero['item_nbr']==item_nbr]
    
    model = sm.OLS.from_formula(formula,data=frame)
    
    r2 = model.fit().rsquared
    
    if not np.isnan(r2):
        mean+=r2
        n+=1
        
        if r2>=0.3:
            print(item_nbr, r2)
    
    results.append(model)

[  5  44  11  85  93   9  45  14 107  52  86  98  82   7  23  95  36  46
 104  59  84 100  16  37  79  73   3 105  25  39  50  77  64  83  69  88
  75  15  91 109   8  41  56  92  49  51  65  13  30   1]
23 0.506585851142
51 0.373154356158


In [25]:
mean/n

0.075314044365546715

In [26]:
results=[0,]
mean = 0
n = 0
print(data['store_nbr'].unique())
for store_nbr in data['store_nbr'].unique():
    store = data[data['store_nbr']==store_nbr]
    
    store_result=[]
    
    for item_nbr in store['item_nbr'].unique():
        
        frame = store[store['item_nbr']==item_nbr]
        
        model = sm.OLS.from_formula(formula,data=frame)
        
        r2 = model.fit().rsquared
        
        if not np.isnan(r2):
            
            if r2>=0.3
                print(store_nbr, item_nbr, r2)
            mean+=r2
            n+=1
        
        store_result.append(model)
        
    results.append(store_result)
    


[ 2  6 38 42  7 13  8 12 43 16 19 21 29 33 30 36]
2 5 0.0262456702298


d:\anaconda\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


2 11 0.0458359707646
2 44 0.0365560285157
2 85 0.218020605943
2 93 0.0663645282834
6 9 0.0202528746617
6 14 0.0474672019245
6 45 0.0345134855461
6 107 0.0453388092174
38 9 0.0491750232671
38 45 0.0573994909263
38 52 0.0182300448955
38 86 0.0553785386226
38 93 0.067188627668
38 98 0.111120265442
42 5 0.0252134883723
42 45 0.0774639235104
42 82 0.0405864828644
7 5 0.0490712022305
7 7 0.0218443930545
7 23 0.043375609305
7 93 0.0784041273424
7 95 0.0995740430863
13 5 0.0499218617447
13 36 0.0585603932047
13 46 0.0476179954609
13 93 0.0645701691375
13 104 0.105161171042
8 5 0.0810882466955
8 23 0.0216062121299
8 59 0.0138820697173
8 84 0.0511082532839
8 93 0.0739625951392
8 100 0.0235550925493
12 16 0.0841545445382
12 37 0.0574378265037
12 79 0.0199817950011
12 86 0.0493217453524
12 93 0.142975580846
43 3 0.104303368209
43 16 0.112190032346
43 37 0.105706161238
43 73 0.0307693978641
43 93 0.0624698221011
43 105 0.0230421082421
16 16 0.109916051021
16 25 0.0482650529755
16 39 0.154904572807


In [27]:
mean/n

0.071204468768795021

In [30]:
results=[0,]
mean = 0
n = 0
print(no_zero['store_nbr'].unique())
for store_nbr in no_zero['store_nbr'].unique():
    store = no_zero[no_zero['store_nbr']==store_nbr]
    
    store_result=[]
    
    print(store_nbr, store['item_nbr'].unique())
    
    for item_nbr in store['item_nbr'].unique():
        
        frame = store[store['item_nbr']==item_nbr]
        
        model = sm.OLS.from_formula(formula,data=frame)
        
        r2 = model.fit().rsquared
        
        if not np.isnan(r2):
            if r2>=0.3:
                print(store_nbr, item_nbr, r2)
            mean+=r2
            n+=1
        
        store_result.append(model)
        
    results.append(store_result)
    


[ 2  6 38 42  7 13  8 12 43 16 19 21 29 33 30 36]
2 [ 5 44 11 85 93]
6 [  9  45  14 107]
38 [ 9 45 52 93 86 98]
42 [ 5 45 82]
7 [ 5  7 23 93 95]
13 [  5  36  46  93 104]
8 [  5  23  59  93  84 100]
12 [16 37 79 93 86]
43 [ 16  37  73  93   3 105]
16 [16 25 39 50 85 93 77 64]
19 [16 83 39 69 93 88 75 15]
21 [  5  44  91 109]
29 [ 8 41 45 56]
33 [ 9 44 85 92 93]
30 [16 44 49 51 65 13]
30 51 0.373154356158
36 [ 9 44 30  1]


In [31]:
mean/n

0.071204468768795021